In [1]:
import numpy as np

data = np.loadtxt( 'C:/Users/Ariadna/Desktop/SED/new/data.txt')



[1.65e-10 1.88e+15 1.42e+15 2.33e+15]


In [2]:
from astropy.table import Table

t = Table([data[:,0],data[:, 1],data[:,2],data[:,3]], names=['energy', 'flux','flux_error_lo','flux_error_hi'])
t['energy'].unit = 'keV'
t['flux'].unit = '1 /(cm2 s keV)'
t['flux_error_lo'].unit = '1/ (cm2 s keV)'
t['flux_error_hi'].unit = '1 / (cm2 s TeV)'
print(t)

  energy          flux          flux_error_lo      flux_error_hi   
   keV      1 / (cm2 keV s)    1 / (cm2 keV s)    1 / (cm2 s TeV)  
---------- ------------------ ------------------ ------------------
  7.36e-11 8140000000000000.0 6010000000000000.0           1.05e+16
  1.65e-10 1880000000000000.0 1420000000000000.0 2330000000000000.0
  4.12e-10  506000000000000.0  349000000000000.0  646000000000000.0
  8.07e-10  153000000000000.0  127000000000000.0  182000000000000.0
  1.64e-09   55000000000000.0   47800000000000.0   63400000000000.0
  3.14e-09   23600000000000.0   21300000000000.0   27200000000000.0
  6.12e-09   10600000000000.0    7890000000000.0   12500000000000.0
  1.07e-08    5990000000000.0    4980000000000.0    6630000000000.0
  2.03e-08    2820000000000.0    2430000000000.0    3870000000000.0
  3.31e-08    1660000000000.0    1130000000000.0    2270000000000.0
       ...                ...                ...                ...
     500.0            8.6e-07           2.26e-07

In [3]:
t.write('formato.ecsv', format='ascii.ecsv')

In [4]:
import numpy as np
from astropy.io import ascii
import naima

data = ascii.read("formato.ecsv")
soft_xray = ascii.read("formato.ecsv")[::5]
vhe = ascii.read("formato.ecsv")

C:\Users\Ariadna\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
print(data)

  energy          flux          flux_error_lo      flux_error_hi   
   keV      1 / (cm2 keV s)    1 / (cm2 keV s)    1 / (cm2 s TeV)  
---------- ------------------ ------------------ ------------------
  7.36e-11 8140000000000000.0 6010000000000000.0           1.05e+16
  1.65e-10 1880000000000000.0 1420000000000000.0 2330000000000000.0
  4.12e-10  506000000000000.0  349000000000000.0  646000000000000.0
  8.07e-10  153000000000000.0  127000000000000.0  182000000000000.0
  1.64e-09   55000000000000.0   47800000000000.0   63400000000000.0
  3.14e-09   23600000000000.0   21300000000000.0   27200000000000.0
  6.12e-09   10600000000000.0    7890000000000.0   12500000000000.0
  1.07e-08    5990000000000.0    4980000000000.0    6630000000000.0
  2.03e-08    2820000000000.0    2430000000000.0    3870000000000.0
  3.31e-08    1660000000000.0    1130000000000.0    2270000000000.0
       ...                ...                ...                ...
     500.0            8.6e-07           2.26e-07

In [11]:
from naima.models import ExponentialCutoffPowerLaw, InverseCompton,Synchrotron

def ElectronSynIC(pars, data):

    # Match parameters to ECPL properties, and give them the appropriate units
    amplitude = 10 ** pars[0] / u.eV
    alpha = pars[1]
    e_cutoff = (10 ** pars[2]) * u.TeV
    B = pars[3] * u.uG

    # Initialize instances of the particle distribution and radiative models
    ECPL = ExponentialCutoffPowerLaw(amplitude, 10.0 * u.TeV, alpha, e_cutoff)
    # Compute IC on CMB and on a FIR component with values from GALPROP for the
    # position of RXJ1713
    IC = InverseCompton(
        ECPL,
        seed_photon_fields=[
            "CMB",
            ["FIR", 26.5 * u.K, 0.415 * u.eV / u.cm ** 3],
        ],
        Eemin=100 * u.GeV,
    )
    SYN = Synchrotron(ECPL, B=B)

    # compute flux at the energies given in data['energy']
    model = IC.flux(data, distance=1.0 * u.kpc) + SYN.flux(
        data, distance=1.0 * u.kpc
    )

    # The first array returned will be compared to the observed spectrum for
    # fitting. All subsequent objects will be stored in the sampler metadata
    # blobs.
    return model, IC.compute_We(Eemin=1 * u.TeV)

In [12]:
from astropy import units as u
def lnprior(pars):
    """
    Return probability of parameter values according to prior knowledge.
    Parameter limits should be done here through uniform prior ditributions
    """
    # Limit norm and B to be positive
    logprob = (
        naima.uniform_prior(pars[0], 0.0, np.inf)
        + naima.uniform_prior(pars[1], -1, 5)
        + naima.uniform_prior(pars[3], 0, np.inf)
    )

    return logprob


if __name__ == "__main__":

    ## Set initial parameters and labels
    # Estimate initial magnetic field and get value in uG
    B0 = 2 * naima.estimate_B(soft_xray, vhe).to("uG").value

    p0 = np.array((33, 2.5, np.log10(48.0), B0))
    labels = ["log10(norm)", "index", "log10(cutoff)", "B"]

    ## Run sampler
    sampler, pos = naima.run_sampler(
        data_table=[soft_xray, vhe],
        p0=p0,
        labels=labels,
        model=ElectronSynIC,
        prior=lnprior,
        nwalkers=32,
        nburn=100,
        nrun=20,
        threads=4,
        prefit=True,
        interactive=False,
    )

    ## Save run results to HDF5 file (can be read later with naima.read_run)
    naima.save_run("test", sampler)

    ## Diagnostic plots
    naima.save_diagnostic_plots(
        "test",
        sampler,
        sed=True,
        blob_labels=["Spectrum", "$W_e$($E_e>1$ TeV)"],
    )
    naima.save_results_table("test", sampler)


INFO: Finding Maximum Likelihood parameters through Nelder-Mead fitting... [naima.core]
INFO:    Initial parameters: [31.97005425  2.5         1.68124124  5.62109578] [naima.core]
INFO:    Initial lnprob(p0): -8522674823183259648.000 [naima.core]
INFO:    New ML parameters : [32.25960103  2.30642651  1.53740582  3.90729515] [naima.core]
INFO:    Maximum lnprob(p0): -252.138 [naima.core]
Burning in the 32 walkers with 100 steps...


KeyboardInterrupt: 